In [1]:
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd

path = 'C:/nh/chromedriver.exe'
driver = webdriver.Chrome(path)

In [12]:
import time

# 찾고자 하는 링크를 드라이버에게 전달 
driver.get('https://www.11st.co.kr/products/1644144364')

# 혹시 모르니 스크롤 부터 조정해서 맨 밑으로 내리고 
SCROLL_PAUSE_SEC = 1
# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [13]:
# 제품명
pname = driver.find_element_by_css_selector('#layBodyWrap > div > div.s_product.s_product_detail > div.l_product_cont_wrap > div > div.l_product_view_wrap > div.l_product_summary > div.l_product_side_info > div.c_product_info_title > h1').text

In [14]:
'''
copy css selector를 했음에도 불구하고 자꾸 요소를 못찾았던 건 사이트 안에 또 다른 사이트가 있어서 그랬던 것. 
그래서 일단 frame을 찾고, 그 프레임 종류들 중에서 리뷰가 있을 것 같은 ifrmReview로 이동
'''
iframes = driver.find_elements_by_css_selector('iframe')
for i in iframes:
    print(i.get_attribute('name'))

prdBbsIfrm
prdDescIfrm
ifrmReview
ifrmQnA
hiddenProcessId



In [15]:
driver.switch_to_frame('ifrmReview')

<ipython-input-15-d698f72d99da>:1: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('ifrmReview')


In [16]:
#review-list-page-area > ul:nth-child(6) > li:nth-child(2)

In [17]:
# 이제 리뷰에서 copy > copy_selector로 해서 나온 요소들로 for문 돌려가면서 찾을 수 있음. 

# 리뷰 더보기 버튼 클릭 횟수 정하기

num = int(driver.find_element_by_css_selector('#review-list-area > div.c_product_title.c_product_title_style3 > h4 > span > i').text)
if num % 20 != 0 :
    click = (num // 20)
else : 
    click = (num // 20)  - 1

# 리뷰 더보기 일단 클릭해서 리뷰창 다 열어놓고 시작
for _ in range(click):
    try : 
        driver.find_element_by_css_selector('#review-list-page-area > div > button').click()
    except:
        from selenium.webdriver.common.keys import Keys
        driver.find_element_by_css_selector('#review-list-page-area > div > button').send_keys(Keys.ENTER)
    time.sleep(1)


# 리뷰 가져오기
# 빈 리스트 생성
id_list = []
pname_list = []
star_list = []
review_list=[]

for i in range(1,click+2):
    for k in range(1,21):
        try : 
            id = driver.find_element_by_css_selector(f'#review-list-page-area > ul:nth-child({i}) > li:nth-child({k}) > dl > dt').text
            id_list.append(id)

            star = driver.find_element_by_css_selector(f'#review-list-page-area > ul:nth-child({i}) > li:nth-child({k}) > div > p.grade > span > em').text
            star_list.append(star)

            try : 
                review = driver.find_element_by_css_selector(f'#review-list-page-area > ul:nth-child({i}) > li:nth-child({k}) > div > div > div.cont_text_wrap > p').text
            except : 
                review = ' '
            review_list.append(review)
            
            pname_list.append(pname)
        except : 
            pass           


In [18]:
# 리뷰 갯수랑 긁어온 갯수 일치하는지 확인
print(f'리뷰 갯수는 {num}개, 긁어온 갯수는 {len(review_list)}개')

리뷰 갯수는 112개, 긁어온 갯수는 112개


In [19]:
import pandas as pd

df = pd.DataFrame({
    'id' : id_list,
    'p_name' : pname_list,
    'star' : star_list,
    'review' : review_list
})

df.head()

,id,p_name,star,review
0,yunj**,[홈플러스]냉장고에쏘옥_유기농쌀_2KG 통,5,자취하는 아들한테 보내준 거예요. 가격도 저렴하고 배송도 빠르고 다 맘에들어요. 담...
1,sung******,[홈플러스]냉장고에쏘옥_유기농쌀_2KG 통,4,용기에 들어있어서 간편하고 맛있어요
2,룰루랄라,[홈플러스]냉장고에쏘옥_유기농쌀_2KG 통,5,남편이랑은 찰현미를 먹어서 아기밥용으로 주문했는데 사이즈가 딱입니다^^ 다음에도 또...
3,shim******,[홈플러스]냉장고에쏘옥_유기농쌀_2KG 통,4,쌀이 똑 떨어진 줄 모르고 있었던 덜랭이 엄마를 당황시키진않네요. 당일배송이라 ㅎㅎ...
4,soul******,[홈플러스]냉장고에쏘옥_유기농쌀_2KG 통,5,홈플에서 수수와 기장을 샀다가 반품하고 다시 다른 제품을 구매하면서 반품 했지만 따...


In [20]:
df.to_csv('C:/nh/11번가_혼합_카카오유기농쌀.csv',encoding='utf-8-sig',index=False)